# Derivativos de Ações e Commodities
##  Precificação com o Método de Diferenças Finitas
<sub>Uirá Caiado. 29 de Junho, 2016<sub>

#### Resumo

_Neste projeto vou implementar o método de diferenças finitas explícito para precificar diferentes derivativos. Como o preço de opções geralmente é descrito por equações diferenciais de difusão ou parabólicas, o método de diferenças finitas se mostra adequado, uma vez que é utilizado justamente para encontrar soluções numéricas para equações diferencias. Depois e implementar o modelo, vou compará-lo com os resultados obtidos pela solução analítica de cada instrumento._

## 1. Introdução

Nesta seção dou uma breve descrição sobre o método de diferenças finitas e declaro o problema que será abordado.

### 1.1. O Método Utilizado

Como colocado por Wilmott, encontrar soluções fechadas para precificação de muitos derivativos pode ser difícil, ou mesmo inviável. Como o preço destes instrumentos frequentemente são descritos por equações diferencias parciais, frequentemente utilizam-se métodos numéricos para encontrar uma solução, como [árvores binomiais](https://en.wikipedia.org/wiki/Binomial_options_pricing_model), simulação de [monte carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) e diferenças finitas.

O método de diferenças finitas encontra o valor de um derivativo calculando-o em todo domínio (factivel) de preços do instrumento base, incluíndo a passagem de tempo até seu vencimento.  Assim, são similares às árvores binomiais. Porém, ao invéz e discretizar os preços do ativo e a passagem do tempo em uma estrutura de árvore, discretiza em um grid.

Diferenças finitas também são muitos boas para lidar com problemas com poucas dimensões e equações diferenciais não lineares (o preço e tempo já são duas, e o Wilmott sugere até quatro). Para muitas dimensões, a implementação começa a ficar complicada e pouco eficiente, coisa que o Monte Carlo lida melhor. Porém, se há exercício antecipado e não linearidade, o método de diferenças finitas acaba se mostrando como solução mais viável.

### 1.2. O Problema

Considerando um ativo-objeto cuja dinâmica do preço satisfaz a seguinte EDE:

$$\frac{\mathrm{d} S_t}{S_t}=\mu\cdot \mathrm{d}t + \sigma\cdot \mathrm{d}W_{t}$$

Deseja-se calcular o preço justo de um derivativo um derivativo com característica europeia cujo payoff é descrito por uma função qualquer $V_T=V(T, S_T)$, onde $T$ é o vencimento do derivativo. $S_T$ é o preço do ativo-objeto em $T$ e é possível negociar qualquer quantidade dele em qualquer instante. Não há custo de transação (corretagem, emolumento, bid-ask spread, etc) e posições vendidas a descoberto no subjacente são permitidas, não havendo custos associados.

Pede-se que se implemente o algoritmo de diferenças finitas explícito e se calcule o (a) preço justo, (b) o Delta e (c) o Gamma de cada instrumento abaixo. Os valores encontrados devem ser comparados com o resultado de suas expressões analítica. A simulação deve ser feita para os payoffs abaixo. $K$ é o Strike da opção.
- $V(T, S_T)=ln(S_T)$
- $V(T, S_T)=(ln(S_T))^2$
- $V(T, S_T)=(S_T-K)^2$
- $V(T, S_T)=\mathbf{1}_{S_T > K}$
- $V(T, S_T)=max(S_T-K, 0)$

##  2. Diferenças Finitas

Abaixo, apresentarei os elementos necessários para implementar o método diferenças finitas, sendo eles: a [diferenciação](https://pt.wikipedia.org/wiki/Diferenciação_numérica) das derivadas parciais necessárias usando o Grid; a discretização da condição final para cada derivativo; e suas respectivas condições de contorno (nos limites do grid).

### 2.1. Diferenciação no Grid

De acordo com Wilmott, o grid utilizado pelo método de diferenças finitas tem passos de tempo e de preço (ou log do preço) geralmente homogênios. Porém, **não há** restrição para a forma do Grid, desde que os ajustes necessários sejam feitos. Como exposto por 

Considerando que podemos discretizar $t$ como $t=T-k\delta t$ e do $S$ como $S = i\delta S$, onde $i$ e $k$ são seus respectivos passos no grid, podemos escrever o valor da opção em cada ponto do grig como sendo:

$$V^{k}_{i} = ( i \delta S, T-k \delta t )$$

Seguindo notas de aula, aplicando Taylor ao preço $V(T, S_T)$ de um derivativo genérico, posso escrever a seguinte equação diferencial parabólica:

$$\frac{\partial V}{\partial t} + a(S, t) \cdot \frac{\partial V^2}{\partial S^2} + b(S, t) \cdot \frac{\partial V}{\partial S} + c(S, t)\cdot V = 0 $$

Sendo que $\frac{\partial V}{\partial t}$ também é chamado de theta ($\theta$), $\frac{\partial V}{\partial S}$ de delta ($\Delta$) e $\frac{\partial V^2}{\partial S^2}$ de gamma ($\Gamma$). Como demostrado por Wilmott, como a definição da primeira derivada $V$ em relação a $t$ é dado por

$$\frac{\partial V}{\partial t} = \underset{h \, \rightarrow \, 0}{\lim}\, \frac{V(S, t + h) - V(S, t)}{h}$$

Então podemos aproximar $\theta$ como sendo $\frac{\partial V}{\partial t} (S, t) \approx \frac{V_{i}^{k} - V_{i}^{k+1} }{\delta t}$. Note que $i$, que é relacionado ao passo do ativo, ficou fixo. *Também estou ignorando os erros de aproximação, como ignorarei em todas as outras diferenciações*.

O mesmo raciocínio pode ser utilizado para aproximar o $\Delta$. Porém, Wilmott ainda sugere que se utilize a [diferença centrada](http://math.stackexchange.com/questions/888259/can-someone-explain-in-general-what-a-central-difference-formula-is-and-what-it), que é dada por $\frac{\partial V}{\partial S} (S, t) \approx \frac{V_{i+1}^{k} - V_{i-1}^{k} }{2 \delta S}$. A discretização anterior também poderia ser utilizada, mas esta oferece um erro de aproximação menor. O único problema desta abordagem é que preciso saber os valores $S + \delta S$ e $S - \delta S$ e nas fronteiras do Grid não terei esta informação. Por tanto, nestes casos utilizarei a discretrização utilizando apenas um dos lados.

Por fim, para achar o $\gamma$, Wilmott subtrai o delta forward do delta backward e divide o resultado por $\delta S$, chegando na aproximação  $\frac{\partial V^2}{\partial S^2} (S, t) \approx \frac{V_{i+1}^{k} - 2 V_{i}^{k} + V_{i-1}^{k} }{\delta S^2}$. Como demonstrado em notas de aula, todos estes resultados podem ser checados aplicando a expansão de Taylor.

### 2.2. Condição Terminal e Payoffs

O valor da opção no vencimento é simplemente seu payoff. Assim, não é necessário resolver nada para $T$, apenas para $S$. Usando a notação de diferenças finitas, os payoffs desejados ficam sendo da forma:

- $V_{i}^{0}=ln(i\delta S)$
- $V_{i}^{0}=(ln(i\delta S))^2$
- $V_{i}^{0}=(i\delta S-K)^2$
- $V_{i}^{0}=\mathbf{1}_{i\delta S > K}$
- $V_{i}^{0}=max(i\delta S-K, 0)$

Como o Wilmott explica, o método de diferenças finitas começa de trás para frente, será destes valores que a iteração começará, como se estivéssemos calculando o preço de um derivativo por árvores binomiais, também de trás para frente.

### 2.3. Condições de Contorno

Quando estivermos percorrendo o Grid, necessitaremos definir o preço do derivativo em seus extremos, quando $S=0$ e $S=I \delta S$, onde $I$ é o ponto mais alto do Grid. Esta condição depende do instrumento que está sendo precificado. Para **todos** dos contratos, utilizaremos a seguinte condição para quando $S=0$:

$$V^{k}_{0} = (1 - r \delta t)V^{k-1}_{0}$$

Para a condição de contorno superior, utilizaremos para a **maioria dos contratos**:

$$V^{k}_{I} = 2V^{k}_{I-1} - V^{k}_{I-2}$$

Esta condição é adequada pois, a medida que $S \rightarrow \infty$, o $\Gamma$ da maioria dos contratos tende a zero. Porém, isso não é verdade para o contrato cujo payoff é descrito por $V_{i}^{0}=(i\delta S-K)^2$. Diferenciando novamente o [delta](https://nbviewer.jupyter.org/github/ucaiado/Replicating_Strategy/blob/master/UiraCaiadoEx01.ipynb) deste contrato, chegamos que o gamma será dado por $2 \cdot e^{(r + \sigma^2)(T-t)}$. Neste caso,
partindo da equação discretizada do $\Gamma$ e resolvendo para $V^{k}_{I}$, tenho que:

\begin{equation}
\begin{aligned}
\Gamma &= \frac{V_{I+1}^{k} - 2 V_{I}^{k} + V_{I-1}^{k} }{\delta S^2}\\
2 \cdot e^{(r + \sigma^2)(\delta t)} &= \frac{V_{I+1}^{k} - 2 V_{I}^{k} + V_{I-1}^{k} }{\delta S^2}\\
V_{I}^{k} &= 2 \delta S^2 e^{(r + \sigma^2)(\delta t)} + 2V_{I-1}^{k} - V_{I-2}^{k}\\
V_{I}^{k} &\approx 2 \delta S^2 + 2V_{I-1}^{k} - V_{I-2}^{k}\\
\end{aligned}
\end{equation}



## 3. Implementando o Modelo

Nesta seção vou decsrever o método de diferenças finitas explícito e implementar os códigos necessários.

### 3.1. Método Explícito

Seguindo notas de aula, substituindo as diferenciações encontradas na equação parabólica mencionada anteriormente e reescrevendo os outros termos com a notação de diferenças finitas, temos que:

\begin{equation}
\begin{aligned}
\frac{\partial V}{\partial t} + a(S, t) \cdot \frac{\partial V^2}{\partial S^2} + b(S, t) \cdot \frac{\partial V}{\partial S} + c(S, t)\cdot V &= 0 \\
\frac{V_{i}^{k} - V_{i}^{k+1} }{\delta t} + a_{i}^{k} \cdot \frac{V_{i+1}^{k} - 2 V_{i}^{k} + V_{i-1}^{k} }{\delta S^2} + \\b_{i}^{k} \cdot \frac{V_{i+1}^{k} - V_{i-1}^{k} }{2 \delta S} + c_{i}^{k}\cdot V_{i}^{k} &= 0
\end{aligned}
\end{equation}

Rearranjando equação acima para isolar $V_{i}^{k+1}$ e renomeando alguns termos, ficamos com:

$$V_{i}^{k+1} = A_{i}^{k}V_{i-1}^{k} + \left( 1 + B_{i}^{k} \right)V_{i}^{k} + C_{i}^{k}V_{i+1}^{k}$$

Onde:

\begin{equation}
\begin{aligned}
A_{i}^{k} &= \nu_1 a_{i}^{k} - 0.5\nu_2 b_{i}^{k} \\
B_{i}^{k} &= -2\nu_1 a_{i}^{k} - \delta t c_{i}^{k} \\
C_{i}^{k} &= \nu_1 a_{i}^{k} + 0.5\nu_2 b_{i}^{k}
\end{aligned}
\end{equation}

Sendo que $\nu_1=\frac{\delta t}{\delta S ^2}$ e $\nu_2=\frac{\delta t}{\delta S}$. A equação acima está definida apenas entre $i=1, ..., I-1$. OS pontos restantes necessários para discretização vem das condições de contorno. Como nós conhecemos o valor terminal em $V_{i}^{0}$, podemos calcular o valor de $V_{i}^{1}$ e assim por diante. Como o valor do instrumento em $k+1$ só depende dos valores dele em $k$, chamamos este método de **método explícito**.

Por último, para que a solução deste método seja estável, é necessário que satisfaça $\delta t \, \leqslant \frac{\delta S^2}{2\sigma^2 S^2}$ e $\delta S \leqslant \frac{2a}{|b|}$.

### 3.2. Criando o Grid

Primeiro, vou definir as classes para a criação do Grid. Como quero acessar os valores posteriormente, vou manter todos em uma estrutura, ainda que este não seja o método mais eficiente aqui.

In [75]:
import numpy as np

class GridNode(object):
    '''
    A representation of a Node of a Grid
    '''
    def __init__(self, k, i):
        '''
        Initialize a GridNode object
        :param k: integer. the time index
        :param i: integer. the asset index
        '''
        # inicia variaveis de controle
        self.k = k
        self.i = i
        self.node_idx = '{:.0f},{:.0f}'.format(k, i)
        # inicia variaveis para precificacao
        self.f_asset_value = 0
        self.f_option_value = 0
        self.f_delta = 0
        self.f_gamma = 0
        self.f_theta = 0
    
    def __str__(self):
        '''
        Return node_idx
        '''
        return self.node_idx

    def __repr__(self):
        '''
        Return the node_idx
        '''
        return self.node_idx

    def __eq__(self, other):
        '''
        Return if a node has different node_idx from the other
        :param other: node object. Node to be compared
        '''
        return self.node_idx == other.node_idx

    def __ne__(self, other):
        '''
        Return if a node has the same node_idx from the other
        :param other: node object. Node to be compared
        '''
        return not self.__eq__(other)

    def __hash__(self):
        '''
        Allow the node object be used as a key in a hash
        table
        '''
        return self.node_idx.__hash__()

        
class Grid(object):
    '''
    A general representation of a Grid to be used by Derivative classes in the
    discretization of their domains
    '''
    def __init__(self, f_vol, f_value, f_time, i_nas):
        '''
        Initialize a Grid object. Save all parameters as attributes
        :param f_vol: float. Volatility of the underlying instrument
        :param f_val: float. The reference value to calculate the grid length 
        :param f_time: float. time to be used in the grid
        :param i_nas: integer. Number of asset steps
        '''
        # inicia variaveis e usa vol para garantir estabilidade
        self.f_nas = 1. * i_nas
        # 'infinito' eh duas vezes o valor
        self.dS = 2 * f_value / self.f_nas
        # como o wilmott garantiu estabilidade
        self.dt = 0.9 / f_vol**2. / self.f_nas**2.
        self.i_nts = int(f_time/self.dt) + 1
        self.dt = f_time / (self.i_nts * 1.)
        # inicia grid. O ponto do tempo inicial eh o final, na verdade
        self.grid = {}
        for k in xrange(int(self.i_nts)):
            for i in xrange(int(self.f_nas)):
                node = GridNode(k, i)
                self.grid[node] = node
    
    def __getitem__(self, k, i):
        '''
        Allow direct access to the nodes of the object
        :param k: integer. the time index
        :param i: integer. the asset index
        '''
        node_idx = GridNode(k, i)
        return self.grid[node_idx]
    
    def __str__(self):
        '''
        A string representation of the node
        '''
        s_aux = ''
        for k in xrange(int(self.i_nts)):
            for i in xrange(int(self.f_nas)):
                valid_node = self.grid[GridNode(k, i)]
                s_aux += '{:7}'.format(valid_node)
            s_aux += '\n'
        return s_aux

In [67]:
x = Grid(f_vol=0.2,
         f_value=100.,
         f_time=1.,
         i_nas=20)

In [72]:
0.9/(0.2**2)/(20.**2)

0.056249999999999994

In [74]:
int(1./0.056249999999999994) + 1

18

In [68]:
x.i_nts

17

In [65]:
x = Grid(f_vol=0.2,
         f_value=100.,
         f_time=1.,
         i_nas=20)
print x

0,0    0,1    0,2    0,3    0,4    0,5    0,6    0,7    0,8    0,9    0,10   0,11   0,12   0,13   0,14   0,15   0,16   0,17   0,18   0,19   
1,0    1,1    1,2    1,3    1,4    1,5    1,6    1,7    1,8    1,9    1,10   1,11   1,12   1,13   1,14   1,15   1,16   1,17   1,18   1,19   
2,0    2,1    2,2    2,3    2,4    2,5    2,6    2,7    2,8    2,9    2,10   2,11   2,12   2,13   2,14   2,15   2,16   2,17   2,18   2,19   
3,0    3,1    3,2    3,3    3,4    3,5    3,6    3,7    3,8    3,9    3,10   3,11   3,12   3,13   3,14   3,15   3,16   3,17   3,18   3,19   
4,0    4,1    4,2    4,3    4,4    4,5    4,6    4,7    4,8    4,9    4,10   4,11   4,12   4,13   4,14   4,15   4,16   4,17   4,18   4,19   
5,0    5,1    5,2    5,3    5,4    5,5    5,6    5,7    5,8    5,9    5,10   5,11   5,12   5,13   5,14   5,15   5,16   5,17   5,18   5,19   
6,0    6,1    6,2    6,3    6,4    6,5    6,6    6,7    6,8    6,9    6,10   6,11   6,12   6,13   6,14   6,15   6,16   6,17   6,18   6,19   
7,0    7,1   

In [53]:
%timeit x = Grid(f_vol=0.1, f_value=10., f_time=100., i_nas=100)

1 loop, best of 3: 4.58 s per loop


Como comentei, não é uma estrtutura eficiente. Demorou quase 5 segundos para criar uma estrtutura de 10.000 nós. Porém acredito que isso me ajudará adiante.

### 3.2. Precificando os instrumentos

Agora preciso precificar os instrumentos. Mas antes vou terminar de implementar o método de diferenças finitas dentro de uma classe básica que será usada para implementar cada instrumento. Já começarei testando para uma Call Européia.

In [55]:
class Derivative(object):
    '''
    A general representation of a Derivative contract.
    '''
    def __init__(self, f_St, f_sigma, f_time, f_r, i_nas, f_K = None):
        '''
        Initialize a Derivative object. Save all parameters as attributes
        :param f_St: float. The price of the underline asset
        :param f_sigma: float. A non negative underline volatility
        :param f_time: float. The time remain until the expiration
        :param f_r: float. The free intereset rate
        :param i_nas: integer. Number of asset steps
        :*param f_K: float. The strike, if applyable
        '''
        # inicia variaveis
        self.s_name = "General"
        self.f_St = f_St
        self.f_K = f_K
        self.f_r = f_r
        self.f_sigma = f_sigma
        self.f_time = f_time
        # inica grid
        self.grid = Grid(f_vol=f_sigma,
                         f_value=f_St,
                         f_time=f_time,
                         i_nas=i_nas)

    def set_final_condition(self):
        '''
        Set up the final condition in the grid, the payoff
        '''
        # apenas o valor final do ativo eh necessario aqui
        for i in xrange(int(self.grid.f_nas)):
            f_S = i * 1. * self.dS 
            self.grid[0, i].f_asset_value = f_S
            self.grid[0, i].f_option_value = self._get_payoff(f_S)

    def go_backwards(self):
        '''
        work backwards in time to calculate the option value
        '''
        # comeco o loop depois do primeiro passo de cada dimensao
        dS = self.grid.dS
        dt = sef.grid.dt
        for k in xrange(1, int(self.grid.i_nts)):
            for i in xrange(1, int(self.grid.f_nas)-1):
                # calcula valores auxiliares
                f_S = i * 1. * self.dS 
                self.grid[k, i].f_asset_value = f_S
                f_V_ip1_km1 = self.grid[k-1, i+1].f_option_value
                f_V_im1_km1 = self.grid[k-1, i-1].f_option_value
                f_V_i_km1 = self.grid[k-1, i].f_option_value
                # calcula gregas por diferenca central
                f_delta = (f_V_ip1_km1 - f_V_im1_km1) / 2. / dS
                f_gamma = (f_V_ip1_km1 - 2 * f_V_i_km1 + f_V_im1_km1) / dS / dS
                # calcula theta 
                f_theta = -0.5 * Vol ^ 2 * S(i) ^ 2 * Gamma
                # guarda as gregas e novo preco
                self.grid[k, i].f_delta = f_delta
                self.grid[k, i].f_gamma = f_gamma
                self.grid[k, i].f_theta = f_theta
                f_option_value = f_V_i_km1 - dt * f_theta
            # aplica condicoes de contorno
            self._apply_boundary_conditions(k)
            # checa exercicio antecipado
            f_aux =  self._early_exercise(f_option_value, f_S)
            self.grid[k, i].f_option_value = f_aux
    
    def _early_exercise(self, f_value, f_S): 
        '''
        Modify the derivative value if it is subject to early exercise
        '''
        return f_value 
    
    def _apply_boundary_conditions(self, k):
        '''
        Apply boundary conditions
        '''
        # para S = 0
        dt = self.grid.dt
        i_nas = int(self.grid.f_nas)
        self.grid[0, k] = self.grid[0, k - 1] * (1 - self.fr * dt)
        # para S=inf
        self.grid[i_nas, k] = 2 * self.grid[i_nas-1, k] * self.grid[i_nas-2, k]
    
    def _get_payoff(self, f_asset_price):
        '''
        Get the payoff of the contract
        :param f_asset_price: float. The base asset price
        '''
        raise NotImplementedError()
    
    def __str__(self):
        '''
        Return a string describing the option
        '''
        s = u"Um(a) {} baseado em um subjacente com preco {:.2f}"
        s += u", {:.1f}% de volatilidade, juros de {:.1f}%"
        if self.f_K:
            s += u", com Strike de {}"
            l_val = [self.s_name, self.f_St, self.f_sigma *100,
                     self.f_r*100, self.f_K, self.f_time,
                     self.f_price, self.f_delta]
        else:
            l_val = [self.s_name, self.f_St, self.f_sigma *100,
                     self.f_r*100, self.f_time, self.f_price,
                     self.f_delta]
        s += u" e vencimento em {:.2f} anos tem o preco de R$ {:.2f} "
        s += u"e Delta de {:.2f}"
        s = s.format(*l_val)
        return s

In [ ]:
class LogContract(Derivative):
    '''
    A representation of a Log Contract
    '''
    def __init__(self, f_St, f_sigma, f_time, f_r, f_K=None):
        '''
        Initialize a LogContract object. Save all parameters as attributes
        :param f_St: float. The price of the underline asset
        :param f_sigma: float. A non negative underline volatility
        :param f_time: float. The time remain until the expiration
        :param f_r: float. The free intereset rate
        :*param f_K: float. The strike, if applyable
        '''
        # inicia variaveis da Derivativo
        super(LogContract, self).__init__(f_St=f_St,
                                          f_sigma=f_sigma,
                                          f_time=f_time,
                                          f_r=f_r,
                                          f_K=None)
        self.s_name = "Contrato Log"
    def _set_price(self):
        '''
        Return the price of the contract
        '''
        exp_r_t = np.exp(-1*self.f_r*self.f_time)
        ln_S = np.log(self.f_St)
        r_var_t = (self.f_r - (self.f_sigma**2)/2) * self.f_time
        ln_S_r_var_t = ln_S + r_var_t
        self.f_price = exp_r_t * ln_S_r_var_t

    def _set_delta(self):
        '''
        Return the delta of the contract
        '''
        exp_r_t = np.exp(-1*self.f_r*self.f_time)
        
        self.f_delta = exp_r_t / self.f_St
        
    def _set_gamma(self):
        '''
        Set the gamma to the contract
        '''
        self.f_delta = exp_r_t / self.f_St

### 3.3. Convergência do método explícito



## 4. Precificando Instrumentos

Nesta seção vou precificar alguns instrumentos e comparar com suas soluçòes analíticas.

### 4.1. Bla

### 4.3. Usando Volatilidade Incerta

## 5. Conclusão

bla bla bla



## 6. Últimas Considerações

Quando ...

_Style notebook and change matplotlib defaults_

In [3]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [4]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("PuBuGn_d", 10))